In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Tous les packages à charger :
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
from sklearn.model_selection import train_test_split
#plus proche voisins
from sklearn.neighbors import KNeighborsClassifier
#importer les métriques pour mesurer les erreurs commises
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
#kmeans
from sklearn.cluster import KMeans
from sklearn import metrics
#foret aléatoire
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
#gradient boosting
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
#On charge les données d'entrainement et de test
train = pd.read_csv("/kaggle/input/ghouls-goblins-and-ghosts-boo/train.csv.zip", header = None) 

In [ ]:
train

In [ ]:
#Pour pouvoir utiliser les données de train comme un array (pour pouvoir faire train[1,2])
tra =train.values
tra[1,2]

#On enlève la première ligne qui contient les noms des colonnes
tra = np.delete(tra, (0), axis=0)
tra

In [ ]:
#On code la dernière colonne des types pour avoir des entier au lieu d'une chaine de caractère
#0 est pour un ghost, 1 pour une Ghoul et 2 pour un Goblin
label_encoder=LabelEncoder()
y=label_encoder.fit_transform(tra[:,6])
y

In [ ]:
#Les données du tableau sont des chaine de caractère et non des entiers
ide = [int(i) for i in tra[:,0]]
bone=[round(float(i),5) for i in tra[:,1]]
rotting=[round(float(i),5) for i in tra[:,2]]
hair =[round(float(i),5) for i in tra[:,3]]
soul=[round(float(i),5) for i in tra[:,4]]

#0 black ; 1 blood ; 2 blue ; 3 clear ; 4 green ; 5 white ; 
color = label_encoder.fit_transform(tra[:,5])

In [ ]:
#Graphique de la répartition des créatures en fonction de la longueur des os et de la décomposition
plt.scatter(bone,rotting,c=y, alpha =0.8)

In [ ]:
#Graphique de la répartition des créatures en fonction de la décomposition et la longueur des cheveux
plt.scatter(rotting,hair,c=y, alpha =0.8)

In [ ]:
#Graphique de la répartition des créatures en fonction de la longueur des cheveux et du pourcentage d'âme
#c'est celui où la répartion "semble" la plus distincte
plt.scatter(hair,soul,c=y, alpha =0.8)

In [ ]:
#Ce graphique ne nous donne pas beaucoup d'infos, les valeurs semblent être très proches et il semble difficile de remarquer 3 groupes
plt.hist2d(bone,hair,cmap='plasma')
plt.xlabel('bone')
plt.ylabel('hair')
plt.colorbar()

In [ ]:
#Il faut rassembler les colonnes pour récupérer toutes les données ensemble
#Pour cela on crée d'abord un tableau de 0
Xtra = np.zeros((371,6))

In [ ]:
#Que l'on rempli avec les valeurs 
for i in range(371):
    Xtra[i,0] = ide[i]
    Xtra[i,1] = bone[i]
    Xtra[i,2]= rotting[i]
    Xtra[i,3]= hair[i]
    Xtra[i,4]= soul[i]
    Xtra[i,5]= color[i]

In [ ]:
train2 = pd.read_csv("/kaggle/input/ghouls-goblins-and-ghosts-boo/train.csv.zip") 
#Depuis le début, le header = none avait transformé mes données du même type que les noms de colonne
#soit des chaines de caractères, d'où le problème pour utiliser les données ...

In [ ]:
sns.pairplot(train2,hue='type') 
#Avec ce graphique, on s'aperçoit que c'est sans doute la longueur des cheveux qui permet le mieux de distinguer 
#les types de monstres, lorsqu'on la compare au pourcentage d'âme

In [ ]:
#J'ai enlevé les colonnes toutes les autres colonnes que celle du type pour ytrain
#et celle de la color qui est une chaine de caractère, de l'identité et du type pour xtrain
ytrain2=train2.drop(['id','bone_length','rotting_flesh','hair_length','color','has_soul'],axis=1)
xtrain2=train2.drop(['id','type','color'],axis=1)

In [ ]:
#On sépare le jeu afin d'avoir un jeu d'entrainement et un jeu de test
Xtrain,Xtest,ytrain,ytest=train_test_split(xtrain2,ytrain2,test_size=0.2,random_state=5)

In [ ]:
Xtrain.shape

In [ ]:
#les données de ytest et y train sont des chaines de caractères, qui vont devenir des entiers
#0 est pour un ghost, 1 pour une Ghoul et 2 pour un Goblin
ytrainchiffre = label_encoder.fit_transform(ytrain.iloc[:,0])
ytestchiffre = label_encoder.fit_transform(ytest.iloc[:,0])

In [ ]:
ytestchiffre

In [ ]:
#Ce graphique permet d'avoir une représentation de nos données pour comparer la répartition du jeu de test et de celui d'entrainement
#Les données comparées sont la longueur des cheveux et le pourcentage d'âme (grace au graphique précédent)
plt.figure()
plt.subplot(121)
plt.scatter(Xtrain.iloc[:,2],Xtrain.iloc[:,3],c=ytrainchiffre,alpha=0.8)
plt.title('Train set')
plt.subplot(122)
plt.scatter(Xtest.iloc[:,2],Xtest.iloc[:,3],c=ytestchiffre,alpha=0.8)
plt.title('Test set')
plt.show()

In [ ]:
#Le premier modèle testé est ici celui des plus proches voisins
model=KNeighborsClassifier(n_neighbors=9)  #on obtient le meilleur score pour une valeur de l'hyperparamètre n=9
#on entraine le modèle et on test son score sur le jeu d'entrainement
model.fit(Xtrain,ytrain)
model.score(Xtrain,ytrain)  

In [ ]:
#Puis sur le jeu de test
model.score(Xtest,ytest)  # On voit ici que le score nous donne plutôt un bon résultat
#(mieux le modèle est mieux on se rapproche de 1)

In [ ]:
#On cherche à prédire les types de notre échantillon à l'aide du modèle
yprob=model.predict(Xtest)

In [ ]:
#Et on compare avec la réalité.
accuracy_score(ytest,yprob)

In [ ]:
confusion_matrix(ytest,yprob) 
#On observe ici quelques erreurs : première colonne correspond aux monstres prédits comme des fantomes
#on a donc 21 qui étaient bien des fantomes, mais 4 qui étaient en réalité des gobelins.
#Cela montre que le modèle n'est pas mauvais mais qu'il commet tout de même des erreurs.

In [ ]:
#La deuxième méthode testée est celle des kmeans
Xtraink,Xtestk,ytraink,ytestk=train_test_split(Xtra,y,test_size=0.2,random_state=5)

In [ ]:
model2=KMeans(n_clusters=3)

In [ ]:
#On entraine le modèle
model2.fit(Xtraink)

In [ ]:
#On prédit des résultats
predictions=model2.predict(Xtraink)

In [ ]:
#on continue à utiliser la longeur des cheveux et le pourcentage d'âme
#La couleur montre ce que notre modèle a prédit
#La forme montre la réalité de la répartition
sns.scatterplot(Xtraink[:,3],Xtraink[:,4],hue=predictions,style=ytraink, palette="deep")   
sns.scatterplot(model2.cluster_centers_[:,3],model2.cluster_centers_[:,4],marker='o', color="r")
plt.title('Clusters avec centroïds')
#Avec ce graphique, on s'aperçoit bien que la méthode kmeans n'est pas adaptée. 
#Les centroids sont trop proches, et on retrouve de nombreuses erreurs entre la prédiction et la réalite
#(idéalement, tout les cercles auraient du être bleu, les croix orange ...)

In [ ]:
silhouette_samples=metrics.silhouette_samples(Xtraink,model2.labels_)
silhouette_score=metrics.silhouette_score(Xtraink,model2.labels_)

In [ ]:
#On calcule le score de la silhouette, qui n'est pas très bon
silhouette_score 

In [ ]:
#On cherche à prédire les types de notre échantillon à l'aide du modèle
yprob2=model2.predict(Xtestk)

In [ ]:
accuracy_score(ytestk,yprob2)
#Le score est ici encore moins bon...

In [ ]:
confusion_matrix(ytestk,yprob2) 
#Cette matrice montre un modèle très loin de la réalité, plus de faux positifs que de positifs...

In [ ]:
#La troisième méthode est celle de la foret aléatoire (cité dans les différents types de classification)
modele3= RandomForestClassifier(n_estimators=100)

In [ ]:
#On entraine
modele3.fit(Xtrain, ytrain)

In [ ]:
#Le score est ici plutôt bon
modele3.score(Xtest,ytest) 

In [ ]:
yprob3=modele3.predict(Xtest)

In [ ]:
accuracy_score(ytest,yprob3)
#C'est le meilleur score depuis le début

In [ ]:
#Comme le montre la matrice de confusion
#Pour 24 fantomes prédis, 20 en sont réellement, 4 n'en sont pas...
confusion_matrix(ytest,yprob3) 

In [ ]:
#La quatrième méthode est celle du Gradient Boosting
modele4= GradientBoostingClassifier()

In [ ]:
modele4.fit(Xtrain, ytrain)

In [ ]:
modele4.score(Xtest,ytest) #Le score est plutôt bon

In [ ]:
yprob4=modele4.predict(Xtest)

In [ ]:
accuracy_score(ytest,yprob4)

In [ ]:
confusion_matrix(ytest,yprob4) 

In [ ]:
#Le modèle le plus adapté semble être celle de la forêt aléatoire
#On va donc utiliser ce dernier pour calculer les types des données à chercher.
#On importe nos données :
test = pd.read_csv("/kaggle/input/ghouls-goblins-and-ghosts-boo/test.csv.zip") 
sample_submission = pd.read_csv("/kaggle/input/ghouls-goblins-and-ghosts-boo/sample_submission.csv.zip") 

In [ ]:
#On supprime la aussi la couleur et l'identité.
xtest2=test.drop(['id','color'],axis=1)

In [ ]:
xtest2

In [ ]:
#On calcule les valeurs prédites avec le modèle.
yprobable=modele3.predict(xtest2)

In [ ]:
yprobable

In [ ]:
sample_submission

In [ ]:
#On les remplace dans le tableau à rendre
for j in range(529):
    sample_submission.iloc[j,1] = yprobable[j]

In [ ]:
sample_submission

In [ ]:
#On transforme ensuite le tableau en fichier csv pour l'envoyer sur Kaggle
sample_submission.to_csv("submission.csv",index=False)